# Implementation

## DeepXDE

In [3]:
'''
First of all we are going to install all the depedencies.
(Matplotlib, Numpy, Scikit Learn, Scikit Optimize and SciPy)
'''
!pip install matplotlib
!pip install numpy
!pip install scikit-learn
!pip install scikit-optimize
!pip install scipy

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


### TensorFlow

In [1]:
'''
Now we need to install from pip the backend library and the backend in this case we will
use Tensorflow2.x and then we will install DeepXDE also from pip.
'''
!pip install upgrade pip
!pip install tensorflow

Defaulting to user installation because normal site-packages is not writeable
ERROR: Could not find a version that satisfies the requirement upgrade (from versions: none)
ERROR: No matching distribution found for upgrade
Defaulting to user installation because normal site-packages is not writeable


In [2]:
'''
Now we will install DeepXDE library
'''
!pip install deepxde

Defaulting to user installation because normal site-packages is not writeable


In [ ]:
import deepxde as dde
from deepxde.backend import tf
import numpy as np
import matplotlib.pyplot as plt

# Definir el dominio rectangular
geom = dde.geometry.Rectangle([0, 0], [1, 2])  # x ∈ [0, 1], y ∈ [0, 2]

# Definir la ecuación de Poisson: ∇²V = 4
def pde(x, y):
    dy_xx = dde.grad.hessian(y, x, i=0, j=0)
    dy_yy = dde.grad.hessian(y, x, i=1, j=1)
    return dy_xx + dy_yy - 4

# Condiciones de frontera
def boundary_x1(x, on_boundary):
    return on_boundary and np.isclose(x[0], 1)

def boundary_x2(x, on_boundary):
    return on_boundary and np.isclose(x[-1], 2)

def boundary_y0(x, on_boundary):
    return on_boundary and np.isclose(x[1], 0)

def boundary_y2(x, on_boundary):
    return on_boundary and np.isclose(x[-1], 2)

# Definir condiciones de frontera (Dirichlet)
bc1 = dde.DirichletBC(geom, lambda x: x[1]**2, boundary_x1)
bc2 = dde.DirichletBC(geom, lambda x: (x[-1] - 1)**2, boundary_x2)
bc3 = dde.DirichletBC(geom, lambda x: x[0]**2, boundary_y0)
bc4 = dde.DirichletBC(geom, lambda x: (x[-1] - 2)**2, boundary_y2)

# Solución exacta para evaluar el error
def exact_solution(x):
    return ((x[:, 0] - x[:, 1])**2).reshape(-1, 1)
# Preparar el problema
data = dde.data.PDE(
    geom,
    pde,
    [bc1, bc2, bc3, bc4],
    num_domain=256,
    num_boundary=64,
    solution=exact_solution,
    num_test=1000,
)

# Red neuronal
net = dde.maps.FNN([2] + [50]*3 + [1], activation="tanh", kernel_initializer="Glorot normal")

# Modelo
model = dde.Model(data, net)

# Entrenamiento
model.compile("adam", lr=1e-3)
losshistory, train_state = model.train(epochs=10000)

# Evaluar el error L2 con la solución exacta
X_test = data.test_points()  # ← Añade los paréntesis aquí
l2_error = dde.metrics.l2_relative_error(
    exact_solution(X_test), model.predict(X_test)
)
print(f"\nError relativo L2: {l2_error:.4e}")

# Graficar el historial de pérdida
dde.utils.plot_loss_history(losshistory)

# (Opcional) Graficar la solución en una cuadrícula
X, Y = np.meshgrid(np.linspace(0, 1, 100), np.linspace(0, 2, 100))
XY = np.hstack((X.flatten()[:, None], Y.flatten()[:, None]))
V_pred = model.predict(XY).reshape(100, 100)

plt.figure(figsize=(6, 5))
plt.contourf(X, Y, V_pred, levels=100, cmap="viridis")
plt.colorbar(label="V(x, y)")
plt.xlabel("x")
plt.ylabel("y")
plt.title("Solución aproximada de V(x, y)")
plt.tight_layout()
plt.show()


Compiling model...
Building feed-forward neural network...
'build' took 0.287184 s

'compile' took 4.538677 s

Training model...

Step      Train loss                                            Test loss                                             Test metric
0         [1.46e+01, 1.11e+00, 2.37e+00, 1.21e-01, 6.59e+00]    [1.46e+01, 1.11e+00, 2.37e+00, 1.21e-01, 6.59e+00]    []  
1000      [6.89e-03, 1.70e-01, 1.63e-01, 9.41e-02, 2.05e+00]    [5.01e-03, 1.70e-01, 1.63e-01, 9.41e-02, 2.05e+00]    []  
2000      [5.41e-03, 1.68e-01, 1.82e-01, 9.40e-02, 2.02e+00]    [3.55e-03, 1.68e-01, 1.82e-01, 9.40e-02, 2.02e+00]    []  
3000      [5.53e-03, 1.60e-01, 1.73e-01, 9.36e-02, 2.02e+00]    [3.55e-03, 1.60e-01, 1.73e-01, 9.36e-02, 2.02e+00]    []  
4000      [3.01e-03, 1.55e-01, 1.79e-01, 8.63e-02, 2.00e+00]    [1.71e-03, 1.55e-01, 1.79e-01, 8.63e-02, 2.00e+00]    []  
5000      [1.28e-03, 1.52e-01, 1.76e-01, 8.22e-02, 2.00e+00]    [8.49e-04, 1.52e-01, 1.76e-01, 8.22e-02, 2.00e+00]    []  
60

### Pytorch